In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm, trange
import time
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Med operation research/Data.xlsx")

In [3]:
df

,Sex,Region,Age,Age(m),Weight,BSA,CCF,NYHA,Diagnosis,Reason,...,Dobutamine,Norepinephrine,Nitro,Isoket,Custodiol,GA(2),АІ(2),PAG(2),EF(2),EDI(2)
0,1,3,11,134.00,38.00,1.32,1,1,1,1,...,1,1,2,1,2,5.0,0.0,17,59,64.0
1,2,4,15,183.13,52.00,1.52,1,1,1,3,...,1,1,2,1,2,3.0,0.5,10,74,52.0
2,2,5,1,10.00,5.85,0.34,2,2,1,1,...,1,1,1,1,2,3.0,0.5,30,30,126.0
3,1,5,13,153.37,42.00,1.37,1,1,1,1,...,1,1,2,1,2,10.0,0.5,35,65,42.0
4,2,3,18,216.63,45.00,1.41,2,2,1,1,...,1,1,2,1,2,5.0,0.5,15,72,53.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,1,2,15,180.00,63.00,1.75,2,2,1,1,...,1,1,2,1,2,9.0,2.0,38,50,116.0
124,1,4,1,4.00,5.20,0.29,2,3,1,1,...,1,2,1,1,2,0.0,0.0,0,94,128.0
125,2,4,3,38.00,17.00,0.68,1,2,2,1,...,1,1,2,1,2,0.0,0.0,0,78,62.6
126,2,4,11,118.00,25.70,0.98,2,3,2,1,...,1,1,2,1,2,10.0,0.5,34,67,66.0


In [4]:
def split_columns(df,
                  criterion_in,
                  criterion_out,
                  drop_criterion = False):
  # P = df.columns[:df.columns.get_loc('z4')+1].tolist()
  P = ['Age(m)', 'Weight', 'EDV', 'DAV', 'dSTJ',
       'AAD', 'DPV', 'DIS', 'DPWV', 'CD', 'ST',
       'Ventilation', 'Exudate', 'Drainage',
       'z1', 'z2', 'z3', 'z4']
  S_in = df.columns[df.columns.get_loc('z4')+1:df
                            .columns.get_loc('EDI(1)')+1].tolist()
  S_out = df.columns[df.columns.get_loc('GA(2)'):].tolist()
  Q_in = [criterion_in]
  Q_out = [criterion_out]
  # U = df.columns[df.columns.get_loc('Dopamine1'):df
  #                           .columns.get_loc('Custodiol')+1].tolist()
  U = ['Dopamine1',	'Dopamine2', 'Dopamine34',	'Dopamine56']
  if drop_criterion:
    S_in.remove(criterion_in)
    S_out.remove(criterion_out)

  F = P+S_in
  X = P+S_in+U
  result_list = [P, S_in, S_out, Q_in, Q_out, U, F, X]

  return result_list 

def func_features(X, F):
  X_beta = X.copy()
  for col in F.columns:
    describe = ['_sin', '_cbrt', '_tan', '_cos', '_square',
                '_cube', '_arctg']

    # X_beta = X_beta.copy()
    # X_beta[f"{col}{describe[0]}"] = np.sin(X_beta[col])  
    X_beta = X_beta.copy()
    # X_beta[f"{col}{describe[1]}"] = np.cbrt(X_beta[col]) 
    X_beta[f"{col}{describe[2]}"] = np.tan(X_beta[col])
    # X_beta[f"{col}{describe[3]}"] = np.cos(X_beta[col])   
    # X_beta[f"{col}{describe[4]}"] = np.square(X_beta[col]) 
    # X_beta[f"{col}{describe[5]}"] = np.power(X_beta[col], 3) 
    # X_beta[f"{col}{describe[6]}"] = np.arctan(X_beta[col]) 
  return X_beta

def mult_u_features(X, Fs, U):
  X_beta = X.copy()
  for col in Fs:
    for u_col in U.columns:
      X_beta = X_beta.copy()
      X_beta[f"{col}*{u_col}"] = X_beta[col]*X_beta[u_col]

  return X_beta

def backward_regression(X, y, 
                        threshold_out,
                        initial_list=[],  
                        verbose=True):
    included=list(X.columns)
    while True:
        changed=False
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included])))
        results_model = model.fit()
        # use all coefs except intercept
        pvalues = results_model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print(f'Drop  with p-value {worst_feature}, {worst_pval}')
        if not changed:
            break

    return included, model, results_model

In [5]:
def get_model(X, F, U, Q_out,
              func_expansion=False,
              u_expansion=True,
              normalize=False,
              threshold_out = 0.1, #0.05 PAG 0.025
              L1_wt = 0, #0
              alpha = 0.005):  # 0.001 PAG 0.0005 EF 0.0005
  if func_expansion:
    X = func_features(X, F)

  if u_expansion:
    Fs = []
    for i in X.columns:
      if i not in U.columns:
        Fs.append(i)

    X = mult_u_features(X, Fs, U)

  if normalize:
    scaler = MinMaxScaler()
    X = pd.DataFrame(scaler.fit_transform(X),
            columns=X.columns, index = X.index) 

  features, model, results_model = backward_regression(X, Q_out, threshold_out)

  results_fr = model.fit_regularized(L1_wt=L1_wt, alpha=alpha, start_params=results_model.params)

  results_fr_fit = sm.regression.linear_model.OLSResults(model, 
                                                         results_fr.params,
                                                         model.normalized_cov_params)
  
  return features, results_model, results_fr_fit

In [6]:
def get_matrix(criterion):
  df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Med operation research/Data.xlsx")

  matrix_lst = split_columns(df, criterion, criterion)

  P = df[matrix_lst[0]]
  S_in = df[matrix_lst[1]]
  S_out = df[matrix_lst[2]]
  Q_in = df[matrix_lst[3]]
  Q_out = df[matrix_lst[4]]
  U = df[matrix_lst[5]]
  F = df[matrix_lst[6]]
  X = df[matrix_lst[7]]

  return X, F, U, Q_out

In [7]:
df[split_columns(df, 'AI(1)', 'AI(1)')[0]]

,Age(m),Weight,EDV,DAV,dSTJ,AAD,DPV,DIS,DPWV,CD,ST,Ventilation,Exudate,Drainage,z1,z2,z3,z4
0,134.00,38.00,58,17,20,28,19,12.0,9.0,22,132,16,21.00,84,-0.30,-0.23,0.38,3.32
1,183.13,52.00,75,25,26,35,20,11.0,14.0,24,0,2,7.30,44,2.97,0.88,2.35,5.30
2,10.00,5.85,72,12,13,14,16,6.0,6.0,18,165,63,55.30,120,2.07,0.97,1.98,2.17
3,153.37,42.00,55,22,26,29,26,13.0,9.0,24,0,8,19.10,90,2.15,0.94,2.75,3.70
4,216.63,45.00,59,18,23,39,22,12.0,12.0,24,60,4,21.30,90,0.12,-0.81,1.53,6.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,180.00,63.00,258,27,39,35,28,14.0,13.0,24,64,19,18.25,85,3.10,0.98,5.50,4.60
124,4.00,5.20,60,11,11,12,13,7.0,7.0,16,648,308,0.00,0,1.83,0.09,1.03,1.16
125,38.00,17.00,72,15,16,18,14,6.0,6.0,19,67,5,27.00,88,1.30,0.90,1.00,1.67
126,118.00,25.70,45,18,25,30,20,6.0,7.0,20,45,4,31.00,160,1.60,1.20,3.70,5.60


In [8]:
X, F, U, Q_out = get_matrix('GA(2)')
GA_features, GA_results_model, GA_results_fr_fit = get_model(X, F, U, Q_out, threshold_out=0.05, alpha=0.001)
X, F, U, Q_out = get_matrix('АІ(2)')
AI_features, AI_results_model, AI_results_fr_fit = get_model(X, F, U, Q_out, threshold_out=0.05, alpha=0.001)
X, F, U, Q_out = get_matrix('PAG(2)')
PAG_features, PAG_results_model, PAG_results_fr_fit = get_model(X, F, U, Q_out, threshold_out=0.02, alpha=0.0008)
X, F, U, Q_out = get_matrix('EF(2)')
EF_features, EF_results_model, EF_results_fr_fit = get_model(X, F, U, Q_out, threshold_out=0.1, alpha=0.001)
X, F, U, Q_out = get_matrix('EDI(2)')
EDI_features, EDI_results_model, EDI_results_fr_fit = get_model(X, F, U, Q_out, threshold_out=0.1, alpha=0.005)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Drop  with p-value EDI(1)*Dopamine1, 0.9836173262201122
Drop  with p-value DPV*Dopamine1, 0.9326395920546577
Drop  with p-value z4, 0.8950990068166511
Drop  with p-value Age(m), 0.8763676944278788
Drop  with p-value z4*Dopamine2, 0.8690595280251606
Drop  with p-value EDV, 0.8072943806129105
Drop  with p-value DIS, 0.8417889278894719
Drop  with p-value Drainage, 0.8690325819877078
Drop  with p-value AI(1)*Dopamine56, 0.8386173302815001
Drop  with p-value Weight, 0.8493384224717466
Drop  with p-value z1*Dopamine34, 0.8371298727981882
Drop  with p-value z1*Dopamine1, 0.7299851289785768
Drop  with p-value DPWV*Dopamine34, 0.6999124046787515
Drop  with p-value z2*Dopamine34, 0.7324158558903666
Drop  with p-value Dopamine34, 0.6473252605933333
Drop  with p-value Ventilation*Dopamine2, 0.5639011422480422
Drop  with p-value GA(1)*Dopamine56, 0.5314566158170244
Drop  with p-value EDI(1)*Dopamine34, 0.5256786427544572
Drop  with p-value DPWV, 0.4475862755878025
Drop  with p-value Ventilation, 0.

In [10]:
GA_results_fr_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  GA(2)   R-squared:                       0.854
Model:                            OLS   Adj. R-squared:                  0.621
Method:                 Least Squares   F-statistic:                     3.669
Date:                Tue, 12 Jul 2022   Prob (F-statistic):           1.76e-06
Time:                        18:49:35   Log-Likelihood:                -235.72
No. Observations:                 128   AIC:                             629.4
Df Residuals:                      49   BIC:                             854.8
Df Model:                          78                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -6.4674      5.715     -1.132      0.263     -17.952       5.017
DAV                       -3.4819      0.988     -3.525      0.001      -5.467      -1.497
dSTJ                       2.8886      0.796      3.628      0.001       1.289       4.488
DPV                       -0.2927      0.105     -2.799      0.007      -0.503      -0.083
CD                         0.7672      0.204      3.763      0.000       0.358       1.177
Exudate                    0.1144      0.069      1.648      0.106      -0.025       0.254
z1                         6.2467      2.130      2.933      0.005       1.967      10.527
z2                         1.7225      0.642      2.681      0.010       0.432       3.013
z3                        -7.1919      2.087     -3.445      0.001     -11.387      -2.997
GA(1)                      0.0592      0.020      2.888      0.006       0.018       0.100
AI(1)                      1.2629      0.719      1.756      0.085      -0.183       2.709
Dopamine1                  4.8153      8.791      0.548      0.586     -12.851      22.481
Dopamine2                 -9.5918      9.909     -0.968      0.338     -29.504      10.320
Dopamine56                11.5469      4.446      2.597      0.012       2.613      20.481
Age(m)*Dopamine1           0.0079      0.026      0.305      0.762      -0.044       0.060
Age(m)*Dopamine2          -0.0179      0.029     -0.619      0.539      -0.076       0.040
Age(m)*Dopamine56          0.0314      0.020      1.586      0.119      -0.008       0.071
Weight*Dopamine56         -0.1073      0.036     -2.985      0.004      -0.180      -0.035
EDV*Dopamine1              0.0267      0.015      1.767      0.083      -0.004       0.057
EDV*Dopamine2             -0.0386      0.020     -1.903      0.063      -0.079       0.002
EDV*Dopamine34             0.0383      0.014      2.784      0.008       0.011       0.066
EDV*Dopamine56            -0.0495      0.017     -2.957      0.005      -0.083      -0.016
DAV*Dopamine1             -0.2383      0.273     -0.872      0.388      -0.788       0.311
DAV*Dopamine2              1.6327      0.478      3.416      0.001       0.672       2.593
DAV*Dopamine34             0.0759      0.241      0.315      0.754      -0.409       0.561
DAV*Dopamine56            -0.6772      0.272     -2.492      0.016      -1.223      -0.131
dSTJ*Dopamine1             1.0471      0.334      3.134      0.003       0.376       1.719
dSTJ*Dopamine2            -1.4709      0.472     -3.118      0.003      -2.419      -0.523
dSTJ*Dopamine34           -1.1595      0.345     -3.358      0.002      -1.853      -0.466
dSTJ*Dopamine56            1.4965      0.461      3.248      0.002       0.571       2.423
AAD*Dopamine1             -0.6449      0.218     -2.953      0.005      -1.084  

In [11]:
m_models = [['GA', len(GA_features), GA_results_fr_fit.rsquared,
             GA_results_fr_fit.rsquared_adj],
            ['AI', len(AI_features), AI_results_fr_fit.rsquared,
              AI_results_fr_fit.rsquared_adj],
            ['PAG', len(PAG_features), PAG_results_fr_fit.rsquared,
              PAG_results_fr_fit.rsquared_adj],
            ['EF', len(EF_features), EF_results_fr_fit.rsquared,
              EF_results_fr_fit.rsquared_adj], 
            ['EDI', len(EDI_features), EDI_results_fr_fit.rsquared,
              EDI_results_fr_fit.rsquared_adj],
            ]

df_models = pd.DataFrame(m_models, columns=['Model name', 'Features',
                                            'R-square', 'Adj R-square'])
df_models

,Model name,Features,R-square,Adj R-square
0,GA,78,0.853821,0.621128
1,AI,74,0.842794,0.623298
2,PAG,93,0.878643,0.546697
3,EF,63,0.752574,0.509015
4,EDI,59,0.764865,0.560851


In [12]:
def get_coded(labels,
              u_lst,
              params_lst):
  params_dict = {par: '' for par in params_lst}
  iter = 1
  for key in params_dict.keys():
    params_dict[key] = f"x{iter}"
    iter += 1

  for key, value in params_dict.items():
    for i in range(len(labels)):
      labels[i] = labels[i].replace(key, value)

  u_dict = {par: '' for par in u_lst}
  iter = 1
  for key in u_dict.keys():
    u_dict[key] = f"u{iter}"
    iter += 1

  for key, value in u_dict.items():
    for i in range(len(labels)):
      labels[i] = labels[i].replace(key, value)

  return labels
      

In [13]:
params_lst = ['Age(m)', 'Weight', 'EDV', 'DAV', 'dSTJ',
              'AAD', 'DPV', 'DIS', 'DPWV', 'CD', 'ST',
              'Ventilation', 'Exudate', 'Drainage',
              'z1', 'z2', 'z3', 'z4', 
              'GA(1)', 'AI(1)', 'PAG(1)', 'EF(1)', 'EDI(1)']

u_cols = ['Dopamine1',	'Dopamine2', 'Dopamine34',	'Dopamine56']

In [14]:
GA_par_coded = get_coded(list(GA_results_model.params.index), list(u_cols), params_lst)
AI_par_coded = get_coded(list(AI_results_model.params.index), list(u_cols), params_lst)
PAG_par_coded = get_coded(list(PAG_results_model.params.index), list(u_cols), params_lst)
EF_par_coded = get_coded(list(EF_results_model.params.index), list(u_cols), params_lst)
EDI_par_coded = get_coded(list(EDI_results_model.params.index), list(u_cols), params_lst)

In [15]:
def get_formula(par, par_coded):
  formula_lst = []
  for i in par_coded:
    formula_lst.append(i)
  
  AI_formula = []

  for i in range(len(formula_lst)):
    AI_formula.append(f"({round(par[i], 4)}*{formula_lst[i]})")
  AI_formula.reverse()
  return AI_formula

In [ ]:
x1 = np.mean(df['Age(m)'])
x2 = np.mean(df['Weight'])
x3 = np.mean(df['EDV'])
x4 = np.mean(df['DAV'])
x5 = np.mean(df['dSTJ'])
x6 = np.mean(df['AAD'])
x7 = np.mean(df['DPV'])
x8 = np.mean(df['DIS'])
x9 = np.mean(df['DPWV'])
x10 = np.mean(df['CD'])
x11 = np.mean(df['ST'])
x12 = np.mean(df['Ventilation'])
x13 = np.mean(df['Exudate'])
x14 = np.mean(df['Drainage'])
x15 = np.mean(df['z1'])
x16 = np.mean(df['z2'])
x17 = np.mean(df['z3'])
x18 = np.mean(df['z4'])
x19 = np.mean(df['GA(1)'])
x20 = np.mean(df['AI(1)'])
x21 = np.mean(df['PAG(1)'])
x22 = np.mean(df['EF(1)'])
x23 = np.mean(df['EDI(1)'])

from sympy import Symbol 
from sympy import *

# x1 = Symbol('x1')
# x2 = Symbol('x2')
# x3 = Symbol('x3')
# x4 = Symbol('x4')
# x5 = Symbol('x5')
# x6 = Symbol('x6')
# x7 = Symbol('x7')
# x8 = Symbol('x8')
# x9 = Symbol('x9')
# x10 = Symbol('x10')
# x11 = Symbol('x11')
# x12 = Symbol('x12')
# x13 = Symbol('x13')
# x14 = Symbol('x14')
# x15 = Symbol('x15')
# x16 = Symbol('x16')
# x17 = Symbol('x17')
# x18 = Symbol('x18')
# x19 = Symbol('x19')
# x20 = Symbol('x20')
# x21 = Symbol('x21')
# x22 = Symbol('x22')
# x23 = Symbol('x23')
# const = Symbol('const')


u1 = Symbol('u1')
u2 = Symbol('u2')
u3 = Symbol('u3')
u4 = Symbol('u4')
u5 = Symbol('u5')
u6 = Symbol('u6')
u7 = Symbol('u7')
u8 = Symbol('u8')
u9 = Symbol('u9')
u10 = Symbol('u10')

# print(f'x1 = {x1}')
# print(f'x2 = {x2}')
# print(f'x3 = {x3}')
# print(f'x4 = {x4}')
# print(f'x5 = {x5}')
# print(f'x6 = {x6}')
# print(f'x7 = {x7}')
# print(f'x8 = {x8}')
# print(f'x9 = {x9}')
# print(f'x10 = {x10}')
# print(f'x11 = {x11}')
# print(f'x12 = {x12}')
# print(f'x13 = {x13}')
# print(f'x14 = {x14}')
# print(f'x15 = {x15}')
# print(f'x16 = {x16}')
# print(f'x17 = {x17}')
# print(f'x18 = {x18}')
# print(f'x19 = {x19}')
# print(f'x20 = {x20}')
# print(f'x21 = {x21}')
# print(f'x22 = {x22}')
# print(f'x23 = {x23}')

In [ ]:
GA_formula = get_formula(GA_results_fr_fit.params, GA_par_coded)
AI_formula = get_formula(AI_results_fr_fit.params, AI_par_coded)
PAG_formula = get_formula(PAG_results_fr_fit.params, PAG_par_coded)
EF_formula = get_formula(EF_results_fr_fit.params, EF_par_coded)
EDI_formula = get_formula(EDI_results_fr_fit.params, EDI_par_coded)

In [ ]:
'+'.join(EDI_formula)

'(-0.1065*x23*u4)+(0.048*x23*u3)+(-0.4433*x22*u4)+(0.1583*x22*u2)+(0.0734*x21*u2)+(2.8613*x20*u4)+(-1.226*x20*u1)+(0.1378*x19*u4)+(-0.1813*x19*u3)+(0.1124*x19*u2)+(-0.0716*x19*u1)+(6.3359*x18*u4)+(-6.7453*x18*u3)+(-1.1678*x18*u2)+(6.0644*x18*u1)+(-6.9028*x17*u4)+(2.2529*x17*u1)+(7.9438*x15*u3)+(-6.5599*x15*u1)+(0.0552*x14*u4)+(-0.0381*x14*u1)+(0.1496*x13*u1)+(-0.2087*x12*u4)+(0.2189*x12*u3)+(-0.0635*x12*u2)+(0.0926*x11*u4)+(-0.1368*x11*u3)+(0.1189*x11*u2)+(-1.9096*x10*u4)+(0.6992*x10*u3)+(-0.631*x10*u1)+(2.2229*x9*u4)+(-0.8532*x9*u1)+(-2.937*x8*u4)+(3.8889*x8*u3)+(-1.6199*x8*u2)+(1.1865*x8*u1)+(0.134*x7*u2)+(-1.9968*x6*u4)+(1.5948*x6*u3)+(-1.9303*x6*u1)+(2.754*x5*u4)+(-1.8002*x5*u1)+(-3.0594*x4*u3)+(4.5656*x4*u1)+(-0.0176*x3*u2)+(0.1133*x2*u1)+(45.1113*u4)+(-14.4073*u3)+(-9.7903*x18)+(9.7805*x15)+(-0.0551*x11)+(1.7071*x10)+(2.6723*x9)+(3.2127*x6)+(3.9532*x5)+(-8.9881*x4)+(0.1458*x3)+(-0.3715*x2)+(10.8927*const)'

**AI criterium**

In [ ]:
(0.0197*x23*u3)+(-0.0213*x23*u1)+(-0.0274*x22*u2)+(0.0406*x22*u1)+(0.0317*x21*u2)+(-0.5033*x20*u3)+(0.7362*x20*u2)+(-0.4157*x20*u1)+(0.0305*x19*u2)+(-0.036*x19*u1)+(-0.5228*x18*u3)+(1.0168*x18*u2)+(-0.6707*x18*u1)+(-0.5872*x17*u4)+(1.0231*x17*u3)+(-1.3636*x17*u2)+(0.756*x17*u1)+(0.4726*x16*u4)+(-0.2898*x16*u2)+(-0.7563*x15*u3)+(1.0427*x15*u2)+(-0.3865*x15*u1)+(0.0079*x14*u4)+(-0.0061*x14*u2)+(-0.0188*x13*u4)+(-0.0149*x13*u3)+(-0.0074*x12*u3)+(0.026*x12*u2)+(-0.0189*x12*u1)+(-0.0028*x11*u4)+(-0.0063*x11*u2)+(0.0113*x11*u1)+(-0.1776*x10*u4)+(0.1257*x10*u3)+(-0.0472*x10*u2)+(0.5598*x9*u4)+(-0.1398*x9*u3)+(-0.1611*x9*u2)+(-0.2625*x8*u4)+(0.0861*x8*u1)+(0.0976*x7*u4)+(-0.2363*x7*u3)+(0.1448*x7*u2)+(-0.0606*x6*u4)+(0.1179*x6*u3)+(-0.3342*x6*u2)+(0.3032*x6*u1)+(0.1617*x5*u4)+(-0.2909*x5*u3)+(0.4927*x5*u2)+(-0.3304*x5*u1)+(0.4161*x4*u3)+(-0.2943*x4*u2)+(-0.0156*x3*u3)+(0.0119*x3*u1)+(-0.0385*x2*u4)+(0.0218*x2*u3)+(-0.0055*x1*u4)+(0.0217*x23)+(-0.0688*x22)+(-0.0965*x21)+(0.0568*x19)+(0.3305*x17)+(0.5829*x16)+(-0.8827*x15)+(0.0811*x13)+(-0.0103*x11)+(0.1032*x10)+(0.3541*x9)+(-0.3786*x8)+(-0.1404*x6)+(0.3635*x4)+(-0.0389*x2)+(-0.008*x1)+(-2.1459)

-0.418495148437501*u1 + 0.5685764078125*u2 - 0.0774558421875003*u3 - 0.353855937499999*u4 + 0.440077787500001

In [ ]:
(0.0197*x23*u3)+(-0.0213*x23*u1)+(-0.0274*x22*u2)+(0.0406*x22*u1)+(0.0317*x21*u2)+(-0.5033*x20*u3)+(0.7362*x20*u2)+(-0.4157*x20*u1)+(0.0305*x19*u2)+(-0.036*x19*u1)+(-0.5228*x18*u3)+(1.0168*x18*u2)+(-0.6707*x18*u1)+(-0.5872*x17*u4)+(1.0231*x17*u3)+(-1.3636*x17*u2)+(0.756*x17*u1)+(0.4726*x16*u4)+(-0.2898*x16*u2)+(-0.7563*x15*u3)\
+(1.0427*x15*u2)+(-0.3865*x15*u1)+(0.0079*x14*u4)+(-0.0061*x14*u2)+(-0.0188*x13*u4)+(-0.0149*x13*u3)+(-0.0074*x12*u3)+(0.026*x12*u2)+(-0.0189*x12*u1)+(-0.0028*x11*u4)+(-0.0063*x11*u2)+(0.0113*x11*u1)+(-0.1776*x10*u4)+(0.1257*x10*u3)+(-0.0472*x10*u2)+(0.5598*x9*u4)+(-0.1398*x9*u3)+(-0.1611*x9*u2)+(-0.2625*x8*u4)+(0.0861*x8*u1)\
+(0.0976*x7*u4)+(-0.2363*x7*u3)+(0.1448*x7*u2)+(-0.0606*x6*u4)+(0.1179*x6*u3)+(-0.3342*x6*u2)+(0.3032*x6*u1)+(0.1617*x5*u4)+(-0.2909*x5*u3)+(0.4927*x5*u2)+(-0.3304*x5*u1)+(0.4161*x4*u3)+(-0.2943*x4*u2)+(-0.0156*x3*u3)+(0.0119*x3*u1)+(-0.0385*x2*u4)+(0.0218*x2*u3)+(-0.0055*x1*u4)+(0.0217*x23)+(-0.0688*x22)+(-0.0965*x21)+(0.0568*x19)+(0.3305*x17)+(0.5829*x16)+(-0.8827*x15)+(0.0811*x13)+(-0.0103*x11)+(0.1032*x10)+(0.3541*x9)+(-0.3786*x8)+(-0.1404*x6)+(0.3635*x4)+(-0.0389*x2)+(-0.008*x1)+(-2.1459)

-0.418495148437501*u1 + 0.5685764078125*u2 - 0.0774558421875003*u3 - 0.353855937499999*u4 + 0.440077787500001

**GA criterium**

In [ ]:
(-0.0254*x23*u4)+(-0.0406*x22*u3)+(-0.0886*x22*u2)+(0.0874*x22*u1)+(0.3396*x21*u4)+(-0.4303*x21*u3)+(0.1634*x21*u1)+(-0.3665*x20*u3)+(1.742*x20*u2)+(-1.8833*x20*u1)+(-0.0313*x19*u3)+(-1.7868*x18*u3)+(1.4119*x18*u1)+(-2.8054*x17*u4)+(2.5671*x17*u3)+(5.7957*x17*u2)+(-4.8645*x17*u1)+(2.283*x16*u4)+(-3.8177*x16*u2)+(3.3661*x16*u1)+(-2.5359*x15*u2)+(0.0156*x14*u4)+(-0.0563*x14*u2)+(0.0438*x14*u1)+(-0.0245*x13*u1)+(-0.073*x12*u4)+(0.1241*x12*u3)+(-0.0604*x12*u1)+(0.0342*x11*u4)+(-0.0622*x11*u3)+(0.0378*x11*u1)+(-0.9537*x10*u4)+(0.7826*x10*u3)+(0.8546*x10*u2)+(-1.1029*x10*u1)+(0.7866*x9*u4)+(0.5785*x9*u2)+(-0.2526*x9*u1)+(-0.837*x8*u4)+(0.6971*x8*u3)+(-1.7835*x8*u2)+(1.3142*x8*u1)+(0.0742*x7*u4)+(-0.2641*x7*u3)+(0.2044*x7*u2)+(-0.1705*x6*u4)+(0.4771*x6*u3)+(0.2322*x6*u2)+(-0.6449*x6*u1)+(1.4965*x5*u4)+(-1.1595*x5*u3)+(-1.4709*x5*u2)+(1.0471*x5*u1)+(-0.6772*x4*u4)+(0.0759*x4*u3)+(1.6327*x4*u2)+(-0.2383*x4*u1)+(-0.0495*x3*u4)+(0.0383*x3*u3)+(-0.0386*x3*u2)+(0.0267*x3*u1)+(-0.1073*x2*u4)+(0.0314*x1*u4)+(-0.0179*x1*u2)+(0.0079*x1*u1)+(11.5469*u4)+(-9.5918*u2)+(4.8153*u1)+(1.2629*x20)+(0.0592*x19)+(-7.1919*x17)+(1.7225*x16)+(6.2467*x15)+(0.1144*x13)+(0.7672*x10)+(-0.2927*x7)+(2.8886*x5)+(-3.4819*x4)+(-6.4674)

0.474984007812499*u1 - 1.7106564796875*u2 + 1.8997319296875*u3 - 1.80876581250001*u4 + 7.8594920328125

In [ ]:
(-0.0254*x23*u4)+(-0.0406*x22*u3)+(-0.0886*x22*u2)+(0.0874*x22*u1)+(0.3396*x21*u4)+(-0.4303*x21*u3)+(0.1634*x21*u1)+(-0.3665*x20*u3)+(1.742*x20*u2)+(-1.8833*x20*u1)+(-0.0313*x19*u3)+(-1.7868*x18*u3)+(1.4119*x18*u1)+(-2.8054*x17*u4)+(2.5671*x17*u3)+(5.7957*x17*u2)+(-4.8645*x17*u1)+(2.283*x16*u4)+(-3.8177*x16*u2)\
+(3.3661*x16*u1)+(-2.5359*x15*u2)+(0.0156*x14*u4)+(-0.0563*x14*u2)+(0.0438*x14*u1)+(-0.0245*x13*u1)+(-0.073*x12*u4)+(0.1241*x12*u3)+(-0.0604*x12*u1)+(0.0342*x11*u4)+(-0.0622*x11*u3)+(0.0378*x11*u1)+(-0.9537*x10*u4)+(0.7826*x10*u3)+(0.8546*x10*u2)+(-1.1029*x10*u1)+(0.7866*x9*u4)+(0.5785*x9*u2)+(-0.2526*x9*u1)+(-0.837*x8*u4)\
+(0.6971*x8*u3)+(-1.7835*x8*u2)+(1.3142*x8*u1)+(0.0742*x7*u4)+(-0.2641*x7*u3)+(0.2044*x7*u2)+(-0.1705*x6*u4)+(0.4771*x6*u3)+(0.2322*x6*u2)+(-0.6449*x6*u1)+(1.4965*x5*u4)+(-1.1595*x5*u3)+(-1.4709*x5*u2)+(1.0471*x5*u1)+(-0.6772*x4*u4)+(0.0759*x4*u3)+(1.6327*x4*u2)+(-0.2383*x4*u1)+(-0.0495*x3*u4)+(0.0383*x3*u3)+(-0.0386*x3*u2)+(0.0267*x3*u1)+(-0.1073*x2*u4)+(0.0314*x1*u4)+(-0.0179*x1*u2)+(0.0079*x1*u1)+(11.5469*u4)+(-9.5918*u2)+(4.8153*u1)+(1.2629*x20)+(0.0592*x19)+(-7.1919*x17)+(1.7225*x16)+(6.2467*x15)+(0.1144*x13)+(0.7672*x10)+(-0.2927*x7)+(2.8886*x5)+(-3.4819*x4)+(-6.4674)

0.474984007812499*u1 - 1.7106564796875*u2 + 1.8997319296875*u3 - 1.80876581250001*u4 + 7.8594920328125

**PAG criterium**

In [ ]:
(-0.266*x23*u4)+(0.2214*x23*u3)+(-0.1758*x23*u1)+(-0.2093*x22*u4)+(-0.966*x21*u3)+(1.4709*x21*u2)+(-0.9762*x21*u1)+(9.2392*x20*u4)+(-7.0033*x20*u3)+(17.8986*x20*u2)+(-17.2228*x20*u1)+(0.1105*x19*u4)+(-0.1734*x19*u3)+(0.5264*x19*u2)+(-0.9144*x19*u1)+(25.3215*x18*u4)+(-12.9425*x18*u3)+(23.2952*x18*u2)+(-13.5744*x18*u1)+(-20.6541*x17*u4)+(13.4439*x17*u3)+(-50.1896*x17*u2)+(41.7728*x17*u1)+(10.4833*x15*u4)+(38.4804*x15*u2)+(-43.2631*x15*u1)+(0.0699*x14*u4)+(0.1205*x14*u3)+(0.6836*x14*u2)+(-0.7942*x14*u1)+(-0.7391*x13*u3)+(-1.5647*x13*u2)+(2.1067*x13*u1)+(-0.3395*x12*u4)+(0.6683*x12*u3)+(-1.1865*x12*u2)+(0.8555*x12*u1)+(0.0567*x11*u4)+(-0.1022*x11*u3)+(0.0561*x11*u2)+(0.051*x11*u1)+(-3.7257*x10*u4)+(2.4936*x10*u3)+(-2.4529*x10*u2)+(2.6853*x10*u1)+(3.0142*x9*u4)+(2.5202*x9*u2)+(-1.7248*x9*u1)+(4.8022*x8*u4)+(2.3317*x8*u3)+(-2.8761*x8*u2)+(2.6134*x8*u1)+(1.4191*x7*u4)+(-1.9471*x7*u3)+(0.8069*x7*u2)+(-0.073*x7*u1)+(-9.4327*x6*u4)+(3.7726*x6*u3)+(-9.4229*x6*u2)+(5.8661*x6*u1)+(8.1518*x5*u4)+(-7.1082*x5*u3)+(22.113*x5*u2)+(-18.2253*x5*u1)+(-3.6302*x4*u4)+(2.0185*x4*u3)+(-15.3492*x4*u2)+(16.1275*x4*u1)+(-0.1714*x3*u2)+(0.1633*x3*u1)+(0.0027*x2*u1)+(0.2326*x1*u4)+(0.1007*x1*u3)+(-0.0139*x1*u2)+(36.7433*u4)+(-9.7506*u2)+(-0.9007*x22)+(0.0909*x21)+(10.3869*x20)+(1.3976*x19)+(-10.5796*x18)+(0.0144*x17)+(3.6668*x16)+(19.3594*x15)+(-0.1702*x14)+(0.9588*x13)+(-0.2023*x11)+(0.7367*x10)+(-3.4989*x8)+(1.077*x7)+(3.9993*x6)+(-7.6468*x4)+(0.265*x3)+(-0.2773)

18.6302017062499*u1 - 25.6631804890625*u2 + 22.710862265625*u3 - 11.6867431968749*u4 + 2.71287869062499

In [ ]:
(-0.266*x23*u4)+(0.2214*x23*u3)+(-0.1758*x23*u1)+(-0.2093*x22*u4)+(-0.966*x21*u3)+(1.4709*x21*u2)+(-0.9762*x21*u1)+(9.2392*x20*u4)+(-7.0033*x20*u3)+(17.8986*x20*u2)+(-17.2228*x20*u1)+(0.1105*x19*u4)+(-0.1734*x19*u3)+(0.5264*x19*u2)+(-0.9144*x19*u1)+(25.3215*x18*u4)+(-12.9425*x18*u3)+(23.2952*x18*u2)+(-13.5744*x18*u1)+(-20.6541*x17*u4)+(13.4439*x17*u3)+(-50.1896*x17*u2)+(41.7728*x17*u1)+(10.4833*x15*u4)+(38.4804*x15*u2)+(-43.2631*x15*u1)+(0.0699*x14*u4)+(0.1205*x14*u3)+(0.6836*x14*u2)+(-0.7942*x14*u1)+(-0.7391*x13*u3)+(-1.5647*x13*u2)+(2.1067*x13*u1)\
+(-0.3395*x12*u4)+(0.6683*x12*u3)+(-1.1865*x12*u2)+(0.8555*x12*u1)+(0.0567*x11*u4)+(-0.1022*x11*u3)+(0.0561*x11*u2)+(0.051*x11*u1)+(-3.7257*x10*u4)+(2.4936*x10*u3)+(-2.4529*x10*u2)+(2.6853*x10*u1)+(3.0142*x9*u4)+(2.5202*x9*u2)+(-1.7248*x9*u1)+(4.8022*x8*u4)+(2.3317*x8*u3)+(-2.8761*x8*u2)+(2.6134*x8*u1)+(1.4191*x7*u4)+(-1.9471*x7*u3)+(0.8069*x7*u2)+(-0.073*x7*u1)+(-9.4327*x6*u4)+(3.7726*x6*u3)+(-9.4229*x6*u2)+(5.8661*x6*u1)+(8.1518*x5*u4)+(-7.1082*x5*u3)+(22.113*x5*u2)+(-18.2253*x5*u1)+(-3.6302*x4*u4)+(2.0185*x4*u3)+(-15.3492*x4*u2)+(16.1275*x4*u1)\
+(-0.1714*x3*u2)+(0.1633*x3*u1)+(0.0027*x2*u1)+(0.2326*x1*u4)+(0.1007*x1*u3)+(-0.0139*x1*u2)+(36.7433*u4)+(-9.7506*u2)+(-0.9007*x22)+(0.0909*x21)+(10.3869*x20)+(1.3976*x19)+(-10.5796*x18)+(0.0144*x17)+(3.6668*x16)+(19.3594*x15)+(-0.1702*x14)+(0.9588*x13)+(-0.2023*x11)+(0.7367*x10)+(-3.4989*x8)+(1.077*x7)+(3.9993*x6)+(-7.6468*x4)+(0.265*x3)+(-0.2773)

18.6302017062499*u1 - 25.6631804890625*u2 + 22.710862265625*u3 - 11.6867431968749*u4 + 2.71287869062499

**EF criterium**

In [ ]:
(0.0904*x23*u4)+(-0.1727*x23*u1)+(-0.8471*x22*u2)+(0.8612*x22*u1)+(0.7882*x21*u2)+(-1.2441*x21*u1)+(2.5689*x20*u2)+(-0.0925*x19*u2)+(0.1527*x19*u1)+(-2.0434*x18*u4)+(-2.4819*x18*u1)+(5.8287*x17*u2)+(-1.7209*x17*u1)+(2.8271*x16*u4)+(-7.812*x16*u2)+(7.1852*x16*u1)+(-2.4878*x15*u4)+(-6.0224*x15*u2)+(4.8997*x15*u1)+(-0.1882*x14*u2)+(0.1792*x14*u1)+(0.1347*x13*u3)+(-0.2918*x13*u1)+(0.2163*x12*u2)+(-0.2222*x12*u1)+(-0.0546*x11*u3)+(0.0597*x11*u1)+(-1.0754*x10*u4)+(3.168*x10*u2)+(-2.4507*x10*u1)+(-0.5817*x9*u2)+(-1.5295*x8*u3)+(0.9776*x8*u1)+(0.171*x7*u4)+(-0.3927*x7*u3)+(0.1718*x7*u1)+(0.7393*x6*u4)+(0.7734*x6*u2)+(-1.0252*x5*u2)+(4.7925*x4*u2)+(-4.0342*x4*u1)+(-0.173*x3*u2)+(0.2348*x3*u1)+(0.1558*x2*u3)+(-0.2881*x2*u2)+(-0.1397*x1*u2)+(0.1265*x1*u1)+(17.9933*u3)+(-44.6678*u2)+(25.8524*u1)+(0.6847*x23)+(1.5536*x21)+(-7.9002*x20)+(-0.1624*x19)+(7.0963*x18)+(-14.9302*x17)+(0.0867*x14)+(0.5451*x13)+(-1.0806*x10)+(-2.1411*x6)+(4.7504*x5)+(-0.4139*x3)+(0.2752*x2)+(22.8423)

-5.43730069375001*u1 + 5.80826536406252*u2 - 2.1135182421875*u3 - 1.6106755453125*u4 + 73.6721019609375

In [ ]:
(0.0904*x23*u4)+(-0.1727*x23*u1)+(-0.8471*x22*u2)+(0.8612*x22*u1)+(0.7882*x21*u2)+(-1.2441*x21*u1)+(2.5689*x20*u2)+(-0.0925*x19*u2)+(0.1527*x19*u1)+(-2.0434*x18*u4)+(-2.4819*x18*u1)+(5.8287*x17*u2)+(-1.7209*x17*u1)+(2.8271*x16*u4)+(-7.812*x16*u2)+(7.1852*x16*u1)+(-2.4878*x15*u4)+(-6.0224*x15*u2)+(4.8997*x15*u1)+(-0.1882*x14*u2)\
+(0.1792*x14*u1)+(0.1347*x13*u3)+(-0.2918*x13*u1)+(0.2163*x12*u2)+(-0.2222*x12*u1)+(-0.0546*x11*u3)+(0.0597*x11*u1)+(-1.0754*x10*u4)+(3.168*x10*u2)+(-2.4507*x10*u1)+(-0.5817*x9*u2)+(-1.5295*x8*u3)+(0.9776*x8*u1)+(0.171*x7*u4)+(-0.3927*x7*u3)+(0.1718*x7*u1)+(0.7393*x6*u4)+(0.7734*x6*u2)+(-1.0252*x5*u2)+(4.7925*x4*u2)+(-4.0342*x4*u1)\
+(-0.173*x3*u2)+(0.2348*x3*u1)+(0.1558*x2*u3)+(-0.2881*x2*u2)+(-0.1397*x1*u2)+(0.1265*x1*u1)+(17.9933*u3)+(-44.6678*u2)+(25.8524*u1)+(0.6847*x23)+(1.5536*x21)+(-7.9002*x20)+(-0.1624*x19)+(7.0963*x18)+(-14.9302*x17)+(0.0867*x14)+(0.5451*x13)+(-1.0806*x10)+(-2.1411*x6)+(4.7504*x5)+(-0.4139*x3)+(0.2752*x2)+(22.8423)

-5.43730069375001*u1 + 5.80826536406252*u2 - 2.1135182421875*u3 - 1.6106755453125*u4 + 73.6721019609375

**EDI criterium**

In [ ]:
(-0.1065*x23*u4)+(0.048*x23*u3)+(-0.4433*x22*u4)+(0.1583*x22*u2)+(0.0734*x21*u2)+(2.8613*x20*u4)+(-1.226*x20*u1)+(0.1378*x19*u4)+(-0.1813*x19*u3)+(0.1124*x19*u2)+(-0.0716*x19*u1)+(6.3359*x18*u4)+(-6.7453*x18*u3)+(-1.1678*x18*u2)+(6.0644*x18*u1)+(-6.9028*x17*u4)+(2.2529*x17*u1)+(7.9438*x15*u3)+(-6.5599*x15*u1)+(0.0552*x14*u4)+(-0.0381*x14*u1)+(0.1496*x13*u1)+(-0.2087*x12*u4)+(0.2189*x12*u3)+(-0.0635*x12*u2)+(0.0926*x11*u4)+(-0.1368*x11*u3)+(0.1189*x11*u2)+(-1.9096*x10*u4)+(0.6992*x10*u3)+(-0.631*x10*u1)+(2.2229*x9*u4)+(-0.8532*x9*u1)+(-2.937*x8*u4)+(3.8889*x8*u3)+(-1.6199*x8*u2)+(1.1865*x8*u1)+(0.134*x7*u2)+(-1.9968*x6*u4)+(1.5948*x6*u3)+(-1.9303*x6*u1)+(2.754*x5*u4)+(-1.8002*x5*u1)+(-3.0594*x4*u3)+(4.5656*x4*u1)+(-0.0176*x3*u2)+(0.1133*x2*u1)+(45.1113*u4)+(-14.4073*u3)+(-9.7903*x18)+(9.7805*x15)+(-0.0551*x11)+(1.7071*x10)+(2.6723*x9)+(3.2127*x6)+(3.9532*x5)+(-8.9881*x4)+(0.1458*x3)+(-0.3715*x2)+(10.8927)

-2.82375173749998*u1 + 7.946095078125*u2 + 0.94546667812498*u3 - 10.70458371875*u4 + 50.6147118984375

In [ ]:
(-0.1065*x23*u4)+(0.048*x23*u3)+(-0.4433*x22*u4)+(0.1583*x22*u2)+(0.0734*x21*u2)+(2.8613*x20*u4)+(-1.226*x20*u1)+(0.1378*x19*u4)+(-0.1813*x19*u3)+(0.1124*x19*u2)+(-0.0716*x19*u1)+(6.3359*x18*u4)+(-6.7453*x18*u3)+(-1.1678*x18*u2)+(6.0644*x18*u1)+(-6.9028*x17*u4)+(2.2529*x17*u1)+(7.9438*x15*u3)+(-6.5599*x15*u1)+(0.0552*x14*u4)\
+(-0.0381*x14*u1)+(0.1496*x13*u1)+(-0.2087*x12*u4)+(0.2189*x12*u3)+(-0.0635*x12*u2)+(0.0926*x11*u4)+(-0.1368*x11*u3)+(0.1189*x11*u2)+(-1.9096*x10*u4)+(0.6992*x10*u3)+(-0.631*x10*u1)+(2.2229*x9*u4)+(-0.8532*x9*u1)+(-2.937*x8*u4)+(3.8889*x8*u3)+(-1.6199*x8*u2)+(1.1865*x8*u1)+(0.134*x7*u2)+(-1.9968*x6*u4)+(1.5948*x6*u3)+(-1.9303*x6*u1)\
+(2.754*x5*u4)+(-1.8002*x5*u1)+(-3.0594*x4*u3)+(4.5656*x4*u1)+(-0.0176*x3*u2)+(0.1133*x2*u1)+(45.1113*u4)+(-14.4073*u3)+(-9.7903*x18)+(9.7805*x15)+(-0.0551*x11)+(1.7071*x10)+(2.6723*x9)+(3.2127*x6)+(3.9532*x5)+(-8.9881*x4)+(0.1458*x3)+(-0.3715*x2)+(10.8927)

-2.82375173749998*u1 + 7.946095078125*u2 + 0.94546667812498*u3 - 10.70458371875*u4 + 50.6147118984375

In [ ]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.2 MB 4.0 MB/s 


In [ ]:
import pulp as p
from pulp import *

Lp_prob = LpProblem('AI_min_optimization', LpMinimize) 

In [ ]:
# u1 = LpVariable("u1", lowBound = 90, upBound = 140)   
# u2 = LpVariable("u2", lowBound = 170, upBound = 400) 

u1 = LpVariable("u2", lowBound = 0, upBound = 10)   
u2 = LpVariable("u3", lowBound = 0, upBound = 10) 
u3 = LpVariable("u5", lowBound = 0, upBound = 9) 
u4 = LpVariable("u6", lowBound = 0, upBound = 8)

In [ ]:
Lp_prob += -0.4185*u1+0.5686*u2-0.0775*u3-0.3535*u4+0.4401

Lp_prob += LpConstraint(-0.4185*u1+0.5686*u2-0.0775*u3-0.3535*u4, LpConstraintGE, 0) , 'AI<='

Lp_prob += LpConstraint(0.4749*u1-1.7107*u2+1.8997*u3-1.8088*u4+7.8596, LpConstraintGE, 0) , 'GA<='
Lp_prob += LpConstraint(0.4749*u1-1.7107*u2+1.8997*u3-1.8088*u4-3.8596, LpConstraintLE, 10), 'GA>='

Lp_prob += LpConstraint(18.6302*u1-25.6632*u2+22.7109*u3-11.6867*u4+2.7129, LpConstraintGE, 0) , 'PAG<='
Lp_prob += LpConstraint(18.6302*u1-25.6632*u2+22.7109*u3-11.6867*u4-8.7129, LpConstraintLE, 10), 'PAG>='

Lp_prob += LpConstraint(-5.4373*u1+5.8083*u2-2.1135*u3-1.6107*u4+18.6721, LpConstraintGE, 55), 'EF<='
Lp_prob += LpConstraint(-5.4373*u1+5.8083*u2-2.1135*u3-1.6107*u4-12.6721, LpConstraintLE, 85), 'EF>='

Lp_prob += LpConstraint(-2.8238*u1+7.9461*u2+0.9455*u3-10.7049*u4+15.6147, LpConstraintGE, 35), 'EDI<='
Lp_prob += LpConstraint(-2.8238*u1+7.9461*u2+0.9455*u3-10.7049*u4-35.6147, LpConstraintLE, 85), 'EDI>='

In [ ]:
Lp_prob

AI_min_optimization:
MINIMIZE
-0.4185*u2 + 0.5686*u3 + -0.0775*u5 + -0.3535*u6 + 0.4401
SUBJECT TO
AI<=: - 0.4185 u2 + 0.5686 u3 - 0.0775 u5 - 0.3535 u6 >= 0

GA<=: 0.4749 u2 - 1.7107 u3 + 1.8997 u5 - 1.8088 u6 >= -7.8596

GA>=: 0.4749 u2 - 1.7107 u3 + 1.8997 u5 - 1.8088 u6 <= 3.8596

PAG<=: 18.6302 u2 - 25.6632 u3 + 22.7109 u5 - 11.6867 u6 >= -2.7129

PAG>=: 18.6302 u2 - 25.6632 u3 + 22.7109 u5 - 11.6867 u6 <= 8.7129

EF<=: - 5.4373 u2 + 5.8083 u3 - 2.1135 u5 - 1.6107 u6 >= -18.6721

EF>=: - 5.4373 u2 + 5.8083 u3 - 2.1135 u5 - 1.6107 u6 <= 12.6721

EDI<=: - 2.8238 u2 + 7.9461 u3 + 0.9455 u5 - 10.7049 u6 >= -15.6147

EDI>=: - 2.8238 u2 + 7.9461 u3 + 0.9455 u5 - 10.7049 u6 <= 35.6147

VARIABLES
u2 <= 10 Continuous
u3 <= 10 Continuous
u5 <= 9 Continuous
u6 <= 8 Continuous

In [ ]:
status = Lp_prob.solve()
p.LpStatus[status]

'Optimal'

In [ ]:
print(p.value(u1), p.value(u2),
      p.value(u3), p.value(u4),
      p.value(Lp_prob.objective))

6.7558976 10.0 9.0 6.1135978 0.44010003210000015
